In [32]:
path = !(echo $(pwd))

In [33]:
import sys
sys.path.insert(0, path[0])

import numpy as np
import chess
import tensorflow
from tensorflow import keras

from utils.board_representation import tensor_to_board
from utils.visualisation import *
from utils.data_loader import load_train_test
from train import sum_squared_loss, pc_column_reg, pc_plane_reg, num_pc_reg, custom_regularized_loss

In [34]:
print(tensorflow.__version__)

2.13.0


In [35]:
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

In [36]:
def visualize_tensor(tensor):

    pieces = ["pawn", "knight", "bishop", "rook", "queen", "king"]
    piece_map = ["white "+piece for piece in pieces] + \
        ["black "+piece for piece in pieces] + \
        ["castling rights", "en passant", "turn"]

    def plot_slice(slice_index):
        plt.figure(figsize=(8, 8))
        plt.imshow(tensor[0][:, :, slice_index], cmap='viridis', vmin=0, vmax=1)
        plt.colorbar()
        plt.gca().invert_yaxis()
        plt.title(f"Slice {slice_index} - {piece_map[slice_index]}")
        plt.show()

    return interact(plot_slice, slice_index=IntSlider(min=0, max=tensor[0].shape[2] - 1, step=1, value=0))

In [37]:
train_data, test_data = load_train_test("./data", "position_tensor_000")
print(len(train_data), len(test_data))

900000 100000


In [38]:
model = keras.models.load_model(
    "./model/20230918231931_autoencoder.keras",
    custom_objects={
        "sum_squared_loss": sum_squared_loss,
        "num_pc_reg": num_pc_reg,
        "pc_column_reg": pc_column_reg,
        "pc_plane_reg": pc_plane_reg,
        "custom_loss": custom_regularized_loss
    }
)

In [39]:
index = 2

In [40]:
sample = test_data[index].reshape((1,8,8,15))

In [41]:
reconstruction = model.predict_on_batch(sample)

In [49]:
model.evaluate(x=[reconstruction], y=[sample], return_dict=True)

1/1 [==============================] - 0s 21ms/step - loss: 1.7422 - sum_squared_loss: 1.5000 - num_pc_reg: 0.0000e+00 - pc_column_reg: 1.4297 - pc_plane_reg: 1.0312


{'loss': 1.7421875,
 'sum_squared_loss': 1.5,
 'num_pc_reg': 0.0,
 'pc_column_reg': 1.4296875,
 'pc_plane_reg': 1.03125}

In [52]:
custom_regularized_loss(sample, reconstruction), sum_squared_loss(sample, reconstruction), num_pc_reg(sample, reconstruction), pc_column_reg(sample, reconstruction), pc_plane_reg(sample, reconstruction)

(<tf.Tensor: shape=(), dtype=bfloat16, numpy=1.25>,
 <tf.Tensor: shape=(), dtype=bfloat16, numpy=1.125>,
 <tf.Tensor: shape=(), dtype=bfloat16, numpy=0.00265503>,
 <tf.Tensor: shape=(), dtype=bfloat16, numpy=0.792969>,
 <tf.Tensor: shape=(), dtype=bfloat16, numpy=0.433594>)

In [44]:
binary_reconstruction = binarize_array(reconstruction)

In [45]:
sample_board = tensor_to_board(sample[0])
reconstructed_board = tensor_to_board(binary_reconstruction[0])

In [46]:
print_board(sample_board)

. . . . . . . .
. . . . . . . K
. . . . . . . .
. . . . . P . .
. . . . . . . k
. . . . . . . .
. . q . . . . .
. . . . . . Q .




In [47]:
print_board(reconstructed_board)

. . . . . . . .
. . . . . . . K
. . . . . . . .
. . . . . P . .
. . . . . . . k
. . . . . . . .
. . q . . . . .
. . . . . . . .




In [48]:
visualize_tensor(abs(reconstruction-sample))

interactive(children=(IntSlider(value=0, description='slice_index', max=14), Output()), _dom_classes=('widget-…

<function __main__.visualize_tensor.<locals>.plot_slice(slice_index)>